# COVID-19 Analysis

In [1]:
import altair as alt
import pandas as pd
import numpy as np
import qgrid
from IPython.display import display_html
import bokeh

from bokeh.models import ColumnDataSource, DataTable, DateFormatter, TableColumn, ColorBar, Range1d, LinearAxis, CategoricalColorMapper,  ColorBar, WheelZoomTool, HoverTool, GlyphRenderer,LabelSet
from bokeh.layouts import row

from datetime import date
from random import randint
import itertools


import numpy as np # we will use this later, so import it now
import pandas as pd
from bokeh.io import output_notebook, show, reset_output, push_notebook
from bokeh.plotting import figure
from bokeh.palettes import Spectral6 as palette

reset_output()
output_notebook()

def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

Loading BokehJS ...

In [2]:
URL_RECOVERED='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
URL_DEATHS='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
URL_CONFIRMED='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'

In [3]:
# Read the data from CSV
df_confirmed = pd.read_csv(URL_CONFIRMED)
df_deaths  = pd.read_csv(URL_DEATHS)
df_recovered  = pd.read_csv(URL_RECOVERED)

# qw = qgrid.show_grid(df_latest_results, show_toolbar=True)

# Display relevant slide of data set
df_first_columns = df_confirmed.iloc[0:10,0:4]
df_last_columns = df_confirmed.iloc[0:10,-10:-1]

display_side_by_side(df_first_columns, df_last_columns)

,Province/State,Country/Region,Lat,Long
0,NaN,Afghanistan,33.0000,65.0000
1,NaN,Albania,41.1533,20.1683
2,NaN,Algeria,28.0339,1.6596
3,NaN,Andorra,42.5063,1.5218
4,NaN,Angola,-11.2027,17.8739
5,NaN,Antigua and Barbuda,17.0608,-61.7964
6,NaN,Argentina,-38.4161,-63.6167
7,NaN,Armenia,40.0691,45.0382
8,Australian Capital Territory,Australia,-35.4735,149.0124
9,New South Wales,Australia,-33.8688,151.2093


In [4]:
# Aggregate province/state information to country level

confirmed_by_country = df_confirmed.groupby('Country/Region').aggregate('sum').reset_index()
confirmed_by_country = confirmed_by_country.drop(columns=['Lat', 'Long'])

deaths_by_country = df_deaths.groupby('Country/Region').aggregate('sum').reset_index()
deaths_by_country = deaths_by_country.drop(columns=['Lat', 'Long'])

recovered_by_country = df_recovered.groupby('Country/Region').aggregate('sum').reset_index()
recovered_by_country = recovered_by_country.drop(columns=['Lat', 'Long'])

In [5]:
df_grouped = pd.concat([confirmed_by_country, deaths_by_country, recovered_by_country], keys=['confirmed', 'deaths', 'recovered'])
df = df_grouped.reset_index()
# Delete column
df = df.drop(columns=['level_1'])
# Rename column
df = df.rename(columns={'level_0': 'status'})

In [6]:
df_country_slice = []
for country in df['Country/Region'].tolist():
    df_sg = df[df['Country/Region'].isin([country])]
    df_sg = df_sg.reset_index()
    df_sg_t = df_sg.transpose()
    # Delete row by index name
    df_sg_t= df_sg_t.drop(["Country/Region", "index"], axis=0)
    # df_sg_t.index = pd.to_datetime(df_sg_t.index)
    df_sg_t.columns = ['confirmed','deaths','recovered']
    df_sg_t.index.names = ['date']
    df_sg_t = df_sg_t.reset_index()
    df_sg_t= df_sg_t.drop([0], axis=0)
    df_sg_t['date']= pd.to_datetime(df_sg_t.date)
    df_sg_t['active'] = df_sg_t['confirmed'] - df_sg_t['deaths'] - df_sg_t['recovered']
    df_sg_t['country']= country
    df_sg_t['confirmed_change']=df_sg_t['confirmed']-df_sg_t['confirmed'].shift(1)
    df_sg_t['death_recovered_change']= -(df_sg_t['deaths'] + df_sg_t['recovered'] - df_sg_t['deaths'].shift(1) - df_sg_t['recovered'].shift(1))
    df_sg_t['change'] = df_sg_t['confirmed_change']+df_sg_t['death_recovered_change']
    df_sg_t= df_sg_t.fillna(0)
    df_country_slice.append(df_sg_t)
df = pd.concat(df_country_slice, ignore_index=True, sort=False)

In [7]:
qw = qgrid.show_grid(df, show_toolbar=True)
qw

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [8]:
df_filtered = df[df.country.isin(['Singapore', 'Germany'])]

alt.Chart(df_filtered).mark_line().encode(
    x='date',
    y='active',
    color='country'
).interactive()

alt.Chart(...)

In [9]:
# Utilize weekly averages
df_w = df.set_index('date')
df_w['active'].resample('W').mean()
# df_w['date'].resample('W')

date
2020-01-26       5.823913
2020-02-02      45.722826
2020-02-09     153.229037
2020-02-16     265.781056
2020-02-23     304.933230
2020-03-01     253.774068
2020-03-08     230.000776
2020-03-15     340.694876
2020-03-22     832.764752
2020-03-29    2122.579193
2020-04-05    4115.530280
2020-04-12    5402.000000
Freq: W-SUN, Name: active, dtype: float64

In [10]:
df_filtered2 = df[df.country.isin(['Singapore', 'Germany', 'China'])]
colors=['orange', 'blue', 'green', 'red']
colors = itertools.cycle(palette)

p = figure(plot_width=800,
           plot_height=500, 
           x_axis_type="datetime",
           tools=['pan', 'wheel_zoom'],
           sizing_mode="scale_width") #, y_axis_type="log")

for i, country in enumerate(df_filtered2['country'].unique()):
    df_chart =df_filtered2[df_filtered2.country == country]
    df_chart= df_chart.sort_values(by=['date'], ascending=True)
    c = next(colors)
    p.line(df_chart['date'], df_chart['active'], legend_label=country, color=c)

p.toolbar.active_drag = 'auto'
p.toolbar.active_scroll =  p.select_one(WheelZoomTool)
p.legend.location = "top_left"

# https://stackoverflow.com/questions/31496628/in-bokeh-how-do-i-add-tooltips-to-a-timeseries-chart-hover-tool
# glyph_renderers = p.select(dict(type=GlyphRenderer))
# bar_source = glyph_renderers[0].data_source
# print(bar_source.data)  #Here we can inspect names to call on hover

# http://docs.bokeh.org/en/1.0.2/docs/user_guide/examples/tools_hover_tooltip_formatting.html
p.add_tools(HoverTool(
    tooltips=[
#          ("Date", "@x{%F}"),
        ("Active cases", "@y{,}"),
    ],
    formatters={
        'Date'      : 'datetime', # use 'datetime' formatter for 'date' field
        'Active cases' : 'printf',   # use 'printf' formatter for 'adj close' field
                                  # use default 'numeral' formatter for other fields
    },
    # display a tooltip whenever the cursor is vertically in line with a glyph
    #     mode='vline'
))
p.legend.click_policy="hide"

# end labels
# labels = LabelSet(x='weight', y='height', text='names', level='glyph',
#               x_offset=5, y_offset=5, source=source, render_mode='canvas')

show(p)

In [15]:
START_COUNTRY="Korea, South"
df_chart = df[df.country == START_COUNTRY]

# Setup size of plot based on min / max values
y1_up=1.1*(df_chart.active.max())
y1_low= 1.1*(df_chart.active.max() / (df_chart.change.max()/df_chart.change.min()))
y2_up= 1.1*(df_chart.change.max())
y2_low= 1.1*(df_chart.change.min())

p = figure(plot_width=800,
           plot_height=500, 
           x_axis_type="datetime",
           tools=['pan', 'wheel_zoom'],
           sizing_mode="scale_width",
           y_range = Range1d(y1_low, y1_up)) #, y_axis_type="log")

# adding two axis
# https://stackoverflow.com/questions/25199665/one-chart-with-two-different-y-axis-ranges-in-bokeh
p.extra_y_ranges = {"change_axis": Range1d(start=y2_low, end=y2_up)}           
p.add_layout(LinearAxis(y_range_name="change_axis"), 'right')

# calculate new column for conditional formatting with CategoricalColorMapper
df['growth']= np.where(df['change']>0, 'pos', 'neg')
# calculate new column to fix display issues with rectangles
df['change_height_adjustment'] = np.where(df['change']>0, df['change']/2, -df['change']/2)

df_chart = df_chart.sort_values(by=['date'], ascending=True)

color_mapper = CategoricalColorMapper(factors=['pos', 'neg'], palette=['red','green'])

data_source = df_chart
p.rect(x='date',
       y='change_height_adjustment', width=5, height='change',source=data_source,
       color= {'field': 'growth', 'transform': color_mapper},
       y_range_name="change_axis", width_units = 'screen')

p.line(df_chart['date'], df_chart['active'], legend_label=START_COUNTRY, color="blue")

p.toolbar.active_drag = 'auto'
p.toolbar.active_scroll =  p.select_one(WheelZoomTool)
p.legend.location = "top_left"
    
show(p)

In [12]:
def curve_by_country(country):
    df_chart = df[df.country == country]

    # Setup size of plot based on min / max values
    y1_up=1.1*(df_chart.active.max())
    y1_low= 1.1*(df_chart.active.max() / (df_chart.change.max()/df_chart.change.min()))
    y2_up= 1.1*(df_chart.change.max())
    y2_low= 1.1*(df_chart.change.min())

    p = figure(plot_width=300,
               plot_height=250, 
               x_axis_type="datetime",
               tools=['pan', 'wheel_zoom'],
               sizing_mode="scale_width",
               y_range = Range1d(y1_low, y1_up)) #, y_axis_type="log")

    # adding two axis
    # https://stackoverflow.com/questions/25199665/one-chart-with-two-different-y-axis-ranges-in-bokeh
    p.extra_y_ranges = {"change_axis": Range1d(start=y2_low, end=y2_up)}           
    p.add_layout(LinearAxis(y_range_name="change_axis"), 'right')

    # calculate new column for conditional formatting with CategoricalColorMapper
    df['growth']= np.where(df['change']>0, 'pos', 'neg')
    # calculate new column to fix display issues with rectangles
    df['change_height_adjustment'] = np.where(df['change']>0, df['change']/2, -df['change']/2)

    df_chart = df_chart.sort_values(by=['date'], ascending=True)

    color_mapper = CategoricalColorMapper(factors=['pos', 'neg'], palette=['red','green'])

    data_source = df_chart
    p.rect(x='date',
           y='change_height_adjustment', width=5, height='change',source=data_source,
           color= {'field': 'growth', 'transform': color_mapper},
           y_range_name="change_axis", width_units = 'screen')

    p.line(df_chart['date'], df_chart['active'], legend_label=country, color="blue")

    p.toolbar.active_drag = 'auto'
    p.toolbar.active_scroll =  p.select_one(WheelZoomTool)
    p.legend.location = "top_left"
    return(p)

In [13]:
countries = ['Singapore', 'China', 'Germany', 'Russia', 'United States']


from bokeh.layouts import gridplot
layout = gridplot([
                  [curve_by_country('Singapore'), curve_by_country('China'), curve_by_country('Korea, South')],
                  [curve_by_country('Austria'), curve_by_country('Diamond Princess'), curve_by_country('Mexico')],
                  [curve_by_country('Peru'), curve_by_country('Switzerland'), curve_by_country('India')]
                 ])
show(layout)

In [14]:
# Countries with negative change

df[df.change < -50].country.unique()

array(['Argentina', 'Australia', 'Austria', 'China', 'Denmark',
       'Diamond Princess', 'Finland', 'France', 'Greece', 'Iran',
       'Korea, South', 'Liechtenstein', 'Luxembourg', 'Malaysia',
       'Mexico', 'Peru', 'Switzerland'], dtype=object)